##**Imports**

In [0]:
# To use torch 0.4
# Also modify loss and model interpolate
#!pip uninstall torch -y
#!pip uninstall torchvision -y
#!pip uninstall fastai -y
#!pip install torchvision==0.2.1
#!pip install torch==0.4.0

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import numpy as np
import os
import cv2

import random
import math
import matplotlib.pyplot as plt
from PIL import Image

import collections
from collections import OrderedDict

In [0]:
#print(torch.__version__)

##**Google Drive Dataset Import**

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##**Model**

In [0]:
class SwitchNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1, using_moving_average=True, last_gamma=False):
        super(SwitchNorm2d, self).__init__()
        self.eps = eps
        self.momentum = momentum
        self.using_moving_average = using_moving_average
        self.last_gamma = last_gamma
        self.weight = nn.Parameter(torch.ones(1, num_features, 1, 1))
        self.bias = nn.Parameter(torch.zeros(1, num_features, 1, 1))
        # determin the weight of the two normalizations to compute the avg mean (will pass through a softmax)
        self.mean_weight = nn.Parameter(torch.ones(2))
        # determin the weight of the two normalizations to compute the avg variance (will pass through a softmax)
        self.var_weight = nn.Parameter(torch.ones(2))
        self.register_buffer('running_mean', torch.zeros(1, num_features, 1))
        self.register_buffer('running_var', torch.zeros(1, num_features, 1))

        self.reset_parameters()

    def reset_parameters(self):
        self.running_mean.zero_()
        self.running_var.zero_()
        if self.last_gamma:
            self.weight.data.fill_(0)
        else:
            self.weight.data.fill_(1)
        self.bias.data.zero_()

    def _check_input_dim(self, input):
        if input.dim() != 4:
            raise ValueError('expected 4D input (got {}D input)'
                             .format(input.dim()))

    def forward(self, x):
        self._check_input_dim(x)
        N, C, H, W = x.size()
        x = x.view(N, C, -1)
        #instancenorm
        mean_in = x.mean(-1, keepdim=True)
        var_in = x.var(-1, keepdim=True)
        #layernorm
        mean_ln = mean_in.mean(1, keepdim=True)
        temp = var_in + mean_in ** 2
        var_ln = temp.mean(1, keepdim=True) - mean_ln ** 2

        if self.training:
            mean_bn = mean_in.mean(0, keepdim=True)
            var_bn = temp.mean(0, keepdim=True) - mean_bn ** 2
            if self.using_moving_average:
                self.running_mean.mul_(1 - self.momentum)
                self.running_mean.add_(self.momentum * mean_bn.data)
                self.running_var.mul_(1 - self.momentum)
                self.running_var.add_(self.momentum * var_bn.data)
            else:
                self.running_mean.add_(mean_bn.data)
                self.running_var.add_(mean_bn.data ** 2 + var_bn.data)
        else:
            mean_bn = torch.autograd.Variable(self.running_mean)
            var_bn = torch.autograd.Variable(self.running_var)

        softmax = nn.Softmax(0)
        mean_weight = softmax(self.mean_weight)
        var_weight = softmax(self.var_weight)

        mean = mean_weight[0] * mean_ln + mean_weight[1] * mean_bn
        var = var_weight[0] * var_ln + var_weight[1] * var_bn
        
        x = (x-mean) / (var+self.eps).sqrt()
        x = x.view(N, C, H, W)
        return x * self.weight + self.bias
      
class conv(nn.Module):
    def __init__(self, num_in_layers, num_out_layers, kernel_size, stride):
        super(conv, self).__init__()
        self.kernel_size = kernel_size
        self.conv_base = nn.Conv2d(num_in_layers, num_out_layers, kernel_size=kernel_size, stride=stride)
        self.normalize = nn.BatchNorm2d(num_out_layers)

    def forward(self, x):
        p = int(np.floor((self.kernel_size-1)/2))
        p2d = (p, p, p, p)
        x = self.conv_base(F.pad(x, p2d))
        x = self.normalize(x)
        return F.elu(x, inplace=True)


class convblock(nn.Module):
    def __init__(self, num_in_layers, num_out_layers, kernel_size):
        super(convblock, self).__init__()
        self.conv1 = conv(num_in_layers, num_out_layers, kernel_size, 1)
        self.conv2 = conv(num_out_layers, num_out_layers, kernel_size, 2)

    def forward(self, x):
        x = self.conv1(x)
        return self.conv2(x)


class maxpool(nn.Module):
    def __init__(self, kernel_size):
        super(maxpool, self).__init__()
        self.kernel_size = kernel_size

    def forward(self, x):
        p = int(np.floor((self.kernel_size-1) / 2))
        p2d = (p, p, p, p)
        return F.max_pool2d(F.pad(x, p2d), self.kernel_size, stride=2)


class resconv(nn.Module):
    def __init__(self, num_in_layers, num_out_layers, stride):
        super(resconv, self).__init__()
        self.num_out_layers = num_out_layers
        self.stride = stride
        self.conv1 = conv(num_in_layers, num_out_layers, 1, 1)
        self.conv2 = conv(num_out_layers, num_out_layers, 3, stride)
        self.conv3 = nn.Conv2d(num_out_layers, 4*num_out_layers, kernel_size=1, stride=1)
        self.conv4 = nn.Conv2d(num_in_layers, 4*num_out_layers, kernel_size=1, stride=stride)
        self.normalize = nn.BatchNorm2d(4*num_out_layers)

    def forward(self, x):
        # do_proj = x.size()[1] != self.num_out_layers or self.stride == 2
        do_proj = True
        shortcut = []
        x_out = self.conv1(x)
        x_out = self.conv2(x_out)
        x_out = self.conv3(x_out)
        if do_proj:
            shortcut = self.conv4(x)
        else:
            shortcut = x
        return F.elu(self.normalize(x_out + shortcut), inplace=True)


class resconv_basic(nn.Module):
    # for resnet18
    def __init__(self, num_in_layers, num_out_layers, stride):
        super(resconv_basic, self).__init__()
        self.num_out_layers = num_out_layers
        self.stride = stride
        self.conv1 = conv(num_in_layers, num_out_layers, 3, stride)
        self.conv2 = conv(num_out_layers, num_out_layers, 3, 1)
        self.conv3 = nn.Conv2d(num_in_layers, num_out_layers, kernel_size=1, stride=stride)
        self.normalize = nn.BatchNorm2d(num_out_layers)

    def forward(self, x):
        #         do_proj = x.size()[1] != self.num_out_layers or self.stride == 2
        do_proj = True
        shortcut = []
        x_out = self.conv1(x)
        x_out = self.conv2(x_out)
        if do_proj:
            shortcut = self.conv3(x)
        else:
            shortcut = x
        return F.elu(self.normalize(x_out + shortcut), inplace=True)


def resblock(num_in_layers, num_out_layers, num_blocks, stride):
    layers = []
    layers.append(resconv(num_in_layers, num_out_layers, stride))
    for i in range(1, num_blocks - 1):
        layers.append(resconv(4 * num_out_layers, num_out_layers, 1))
    layers.append(resconv(4 * num_out_layers, num_out_layers, 1))
    return nn.Sequential(*layers)


def resblock_basic(num_in_layers, num_out_layers, num_blocks, stride):
    layers = []
    layers.append(resconv_basic(num_in_layers, num_out_layers, stride))
    for i in range(1, num_blocks):
        layers.append(resconv_basic(num_out_layers, num_out_layers, 1))
    return nn.Sequential(*layers)


class upconv(nn.Module):
    def __init__(self, num_in_layers, num_out_layers, kernel_size, scale):
        super(upconv, self).__init__()
        self.scale = scale
        self.conv1 = conv(num_in_layers, num_out_layers, kernel_size, 1)

    def forward(self, x):
        x = nn.functional.interpolate(x, scale_factor=self.scale, mode='bilinear', align_corners=True)
        #x = nn.functional.upsample(x, scale_factor=self.scale, mode='bilinear', align_corners=True)
        return self.conv1(x)


class get_disp(nn.Module):
    def __init__(self, num_in_layers):
        super(get_disp, self).__init__()
        self.conv1 = nn.Conv2d(num_in_layers, 2, kernel_size=3, stride=1)
        self.normalize = nn.BatchNorm2d(2)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        p = 1
        p2d = (p, p, p, p)
        x = self.conv1(F.pad(x, p2d))
        x = self.normalize(x)
        return 0.3 * self.sigmoid(x)

In [0]:
class Resnet50_md(nn.Module):
    def __init__(self, num_in_layers):
        super(Resnet50_md, self).__init__()
        # encoder
        self.conv1 = conv(num_in_layers, 64, 7, 2)  # H/2  -   64D
        self.pool1 = maxpool(3)  # H/4  -   64D
        self.conv2 = resblock(64, 64, 3, 2)  # H/8  -  256D
        self.conv3 = resblock(256, 128, 4, 2)  # H/16 -  512D
        self.conv4 = resblock(512, 256, 6, 2)  # H/32 - 1024D
        self.conv5 = resblock(1024, 512, 3, 2)  # H/64 - 2048D

        # decoder
        self.upconv6 = upconv(2048, 512, 3, 2)
        self.iconv6 = conv(1024 + 512, 512, 3, 1)

        self.upconv5 = upconv(512, 256, 3, 2)
        self.iconv5 = conv(512+256, 256, 3, 1)

        self.upconv4 = upconv(256, 128, 3, 2)
        self.iconv4 = conv(256+128, 128, 3, 1)
        self.disp4_layer = get_disp(128)

        self.upconv3 = upconv(128, 64, 3, 2)
        self.iconv3 = conv(64+64+2, 64, 3, 1)
        self.disp3_layer = get_disp(64)

        self.upconv2 = upconv(64, 32, 3, 2)
        self.iconv2 = conv(32+64+2, 32, 3, 1)
        self.disp2_layer = get_disp(32)

        self.upconv1 = upconv(32, 16, 3, 2)
        self.iconv1 = conv(16+2, 16, 3, 1)
        self.disp1_layer = get_disp(16)

        #for m in self.modules():
        #    if isinstance(m, nn.Conv2d):
        #        nn.init.xavier_uniform_(m.weight)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, 0.01)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def copy_meta_weights(self, meta_weights):
        for k, p in self.named_parameters():
            if p.requires_grad and k in meta_weights.keys():
                p.data = meta_weights[k].data.clone()
    
    def copy_meta_weights_grad(self, meta_weights):
        for k, p in self.named_parameters():
            if p.requires_grad and k in meta_weights.keys():
                p = meta_weights[k]

    def get_learnable_params(self):
        params = OrderedDict()
        for k, p in self.named_parameters():
            if p.requires_grad:
                params[k] = p
        return params

    def forward(self, x, bn_update):
        # encoder
        x1 = self.conv1(x)
        x_pool1 = self.pool1(x1)
        x2 = self.conv2(x_pool1)
        x3 = self.conv3(x2)
        
        # test to skip decoder update in the forward pass for the bn update 
        if bn_update:
            return x
        
        x4 = self.conv4(x3)
        x5 = self.conv5(x4)
        
        # skips
        skip1 = x1
        skip2 = x_pool1
        skip3 = x2
        skip4 = x3
        skip5 = x4

        # decoder
        upconv6 = self.upconv6(x5)
        concat6 = torch.cat((upconv6, skip5), 1)
        iconv6 = self.iconv6(concat6)

        upconv5 = self.upconv5(iconv6)
        concat5 = torch.cat((upconv5, skip4), 1)
        iconv5 = self.iconv5(concat5)

        upconv4 = self.upconv4(iconv5)
        concat4 = torch.cat((upconv4, skip3), 1)
        iconv4 = self.iconv4(concat4)
        self.disp4 = self.disp4_layer(iconv4)
        self.udisp4 = nn.functional.interpolate(self.disp4, scale_factor=2, mode='bilinear', align_corners=True)
        #self.udisp4 = nn.functional.upsample(self.disp4, scale_factor=2, mode='bilinear', align_corners=True)

        upconv3 = self.upconv3(iconv4)
        concat3 = torch.cat((upconv3, skip2, self.udisp4), 1)
        iconv3 = self.iconv3(concat3)
        self.disp3 = self.disp3_layer(iconv3)
        self.udisp3 = nn.functional.interpolate(self.disp3, scale_factor=2, mode='bilinear', align_corners=True)
        #self.udisp3 = nn.functional.upsample(self.disp3, scale_factor=2, mode='bilinear', align_corners=True)

        upconv2 = self.upconv2(iconv3)
        concat2 = torch.cat((upconv2, skip1, self.udisp3), 1)
        iconv2 = self.iconv2(concat2)
        self.disp2 = self.disp2_layer(iconv2)
        self.udisp2 = nn.functional.interpolate(self.disp2, scale_factor=2, mode='bilinear', align_corners=True)
        #self.udisp2 = nn.functional.upsample(self.disp2, scale_factor=2, mode='bilinear', align_corners=True)

        upconv1 = self.upconv1(iconv2)
        concat1 = torch.cat((upconv1, self.udisp2), 1)
        iconv1 = self.iconv1(concat1)
        self.disp1 = self.disp1_layer(iconv1)
        return self.disp1, self.disp2, self.disp3, self.disp4

In [0]:
class Resnet50_md_switchnorm(nn.Module):
    def __init__(self, num_in_layers):
        super(Resnet50_md, self).__init__()
        # encoder
        self.conv1 = conv(num_in_layers, 64, 7, 2)  # H/2  -   64D
        self.pool1 = maxpool(3)  # H/4  -   64D
        self.conv2 = resblock(64, 64, 3, 2)  # H/8  -  256D
        self.conv3 = resblock(256, 128, 4, 2)  # H/16 -  512D
        self.conv4 = resblock(512, 256, 6, 2)  # H/32 - 1024D
        self.conv5 = resblock(1024, 512, 3, 2)  # H/64 - 2048D

        # decoder
        self.upconv6 = upconv(2048, 512, 3, 2)
        self.iconv6 = conv(1024 + 512, 512, 3, 1)

        self.upconv5 = upconv(512, 256, 3, 2)
        self.iconv5 = conv(512+256, 256, 3, 1)

        self.upconv4 = upconv(256, 128, 3, 2)
        self.iconv4 = conv(256+128, 128, 3, 1)
        self.disp4_layer = get_disp(128)

        self.upconv3 = upconv(128, 64, 3, 2)
        self.iconv3 = conv(64+64+2, 64, 3, 1)
        self.disp3_layer = get_disp(64)

        self.upconv2 = upconv(64, 32, 3, 2)
        self.iconv2 = conv(32+64+2, 32, 3, 1)
        self.disp2_layer = get_disp(32)

        self.upconv1 = upconv(32, 16, 3, 2)
        self.iconv1 = conv(16+2, 16, 3, 1)
        self.disp1_layer = get_disp(16)

        #for m in self.modules():
        #    if isinstance(m, nn.Conv2d):
        #        nn.init.xavier_uniform_(m.weight)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, 0.01)
            elif isinstance(m, SwitchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def copy_meta_weights(self, meta_weights):
        for k, p in self.named_parameters():
            if p.requires_grad and k in meta_weights.keys():
                p.data = meta_weights[k].data.clone()
    
    def copy_meta_weights_grad(self, meta_weights):
        for k, p in self.named_parameters():
            if p.requires_grad and k in meta_weights.keys():
                p = meta_weights[k]

    def get_learnable_params(self):
        params = OrderedDict()
        for k, p in self.named_parameters():
            if p.requires_grad:
                params[k] = p
        return params

    def forward(self, x, bn_update):
        # encoder
        x1 = self.conv1(x)
        x_pool1 = self.pool1(x1)
        x2 = self.conv2(x_pool1)
        x3 = self.conv3(x2)
        x4 = self.conv4(x3)
        x5 = self.conv5(x4)

        # test to skip decoder update in the forward pass for the bn update 
        if bn_update:
            return x

        # skips
        skip1 = x1
        skip2 = x_pool1
        skip3 = x2
        skip4 = x3
        skip5 = x4

        # decoder
        upconv6 = self.upconv6(x5)
        concat6 = torch.cat((upconv6, skip5), 1)
        iconv6 = self.iconv6(concat6)

        upconv5 = self.upconv5(iconv6)
        concat5 = torch.cat((upconv5, skip4), 1)
        iconv5 = self.iconv5(concat5)

        upconv4 = self.upconv4(iconv5)
        concat4 = torch.cat((upconv4, skip3), 1)
        iconv4 = self.iconv4(concat4)
        self.disp4 = self.disp4_layer(iconv4)
        self.udisp4 = nn.functional.interpolate(self.disp4, scale_factor=2, mode='bilinear', align_corners=True)

        upconv3 = self.upconv3(iconv4)
        concat3 = torch.cat((upconv3, skip2, self.udisp4), 1)
        iconv3 = self.iconv3(concat3)
        self.disp3 = self.disp3_layer(iconv3)
        self.udisp3 = nn.functional.interpolate(self.disp3, scale_factor=2, mode='bilinear', align_corners=True)

        upconv2 = self.upconv2(iconv3)
        concat2 = torch.cat((upconv2, skip1, self.udisp3), 1)
        iconv2 = self.iconv2(concat2)
        self.disp2 = self.disp2_layer(iconv2)
        self.udisp2 = nn.functional.interpolate(self.disp2, scale_factor=2, mode='bilinear', align_corners=True)

        upconv1 = self.upconv1(iconv2)
        concat1 = torch.cat((upconv1, self.udisp2), 1)
        iconv1 = self.iconv1(concat1)
        self.disp1 = self.disp1_layer(iconv1)
        return self.disp1, self.disp2, self.disp3, self.disp4

##**Loss**

In [0]:
class MonodepthLoss(nn.modules.Module):
    def __init__(self, n=4, SSIM_w=0.85, disp_gradient_w=1.0, lr_w=1.0):
        super(MonodepthLoss, self).__init__()
        self.SSIM_w = SSIM_w
        self.disp_gradient_w = disp_gradient_w
        self.lr_w = lr_w
        self.n = n

    def scale_pyramid(self, img, num_scales):
        scaled_imgs = [img]
        s = img.size()
        h = s[2]
        w = s[3]
        for i in range(num_scales - 1):
            ratio = 2 ** (i + 1)
            nh = h // ratio
            nw = w // ratio
            scaled_imgs.append(nn.functional.interpolate(img, size=[nh, nw], mode='bilinear', align_corners=True))
            #scaled_imgs.append(nn.functional.upsample(img, size=[nh, nw], mode='bilinear', align_corners=True))
        return scaled_imgs

    def gradient_x(self, img):
        # Pad input to keep output size consistent
        img = F.pad(img, (0, 1, 0, 0), mode='replicate')
        gx = img[:, :, :, :-1] - img[:, :, :, 1:]  # NCHW
        return gx

    def gradient_y(self, img):
        # Pad input to keep output size consistent
        img = F.pad(img, (0, 0, 0, 1), mode='replicate')
        gy = img[:, :, :-1, :] - img[:, :, 1:, :]  # NCHW
        return gy

    def apply_disparity(self, img, disp):
        batch_size, _, height, width = img.size()

        # Original coordinates of pixels
        x_base = torch.linspace(0, 1, width).repeat(batch_size, height, 1).type_as(img)
        y_base = torch.linspace(0, 1, height).repeat(batch_size, width, 1).transpose(1, 2).type_as(img)

        # Apply shift in X direction
        x_shifts = disp[:, 0, :, :]  # Disparity is passed in NCHW format with 1 channel
        flow_field = torch.stack((x_base + x_shifts, y_base), dim=3)
        # In grid_sample coordinates are assumed to be between -1 and 1
        output = F.grid_sample(img, 2*flow_field - 1, mode='bilinear', padding_mode='zeros')

        return output

    def generate_image_left(self, img, disp):
        return self.apply_disparity(img, -disp)

    def generate_image_right(self, img, disp):
        return self.apply_disparity(img, disp)

    def SSIM(self, x, y):
        C1 = 0.01 ** 2
        C2 = 0.03 ** 2

        mu_x = nn.AvgPool2d(3, 1)(x)
        mu_y = nn.AvgPool2d(3, 1)(y)
        mu_x_mu_y = mu_x * mu_y
        mu_x_sq = mu_x.pow(2)
        mu_y_sq = mu_y.pow(2)

        sigma_x = nn.AvgPool2d(3, 1)(x * x) - mu_x_sq
        sigma_y = nn.AvgPool2d(3, 1)(y * y) - mu_y_sq
        sigma_xy = nn.AvgPool2d(3, 1)(x * y) - mu_x_mu_y

        SSIM_n = (2 * mu_x_mu_y + C1) * (2 * sigma_xy + C2)
        SSIM_d = (mu_x_sq + mu_y_sq + C1) * (sigma_x + sigma_y + C2)
        SSIM = SSIM_n / SSIM_d

        return torch.clamp((1 - SSIM) / 2, 0, 1)

    def disp_smoothness(self, disp, pyramid):
        disp_gradients_x = [self.gradient_x(d) for d in disp]
        disp_gradients_y = [self.gradient_y(d) for d in disp]

        image_gradients_x = [self.gradient_x(img) for img in pyramid]
        image_gradients_y = [self.gradient_y(img) for img in pyramid]

        weights_x = [torch.exp(-torch.mean(torch.abs(g), 1, keepdim=True)) for g in image_gradients_x]
        weights_y = [torch.exp(-torch.mean(torch.abs(g), 1, keepdim=True)) for g in image_gradients_y]

        smoothness_x = [disp_gradients_x[i] * weights_x[i] for i in range(self.n)]
        smoothness_y = [disp_gradients_y[i] * weights_y[i] for i in range(self.n)]

        return [torch.abs(smoothness_x[i]) + torch.abs(smoothness_y[i]) for i in range(self.n)]

    def forward(self, input, target):
        '''
        Args:
            input [disp1, disp2, disp3, disp4]
            target [left, right]

        Return:
            (float): The loss
        '''
        left, right = target
        left_pyramid = self.scale_pyramid(left, self.n)
        right_pyramid = self.scale_pyramid(right, self.n)

        # Prepare disparities
        disp_left_est = [d[:, 0, :, :].unsqueeze(1) for d in input]
        disp_right_est = [d[:, 1, :, :].unsqueeze(1) for d in input]

        self.disp_left_est = disp_left_est
        self.disp_right_est = disp_right_est
        # Generate images
        left_est = [self.generate_image_left(right_pyramid[i], disp_left_est[i]) for i in range(self.n)]
        right_est = [self.generate_image_right(left_pyramid[i], disp_right_est[i]) for i in range(self.n)]
        self.left_est = left_est
        self.right_est = right_est

        # L-R Consistency
        right_left_disp = [self.generate_image_left(disp_right_est[i], disp_left_est[i]) for i in range(self.n)]
        left_right_disp = [self.generate_image_right(disp_left_est[i], disp_right_est[i]) for i in range(self.n)]

        # Disparities smoothness
        disp_left_smoothness = self.disp_smoothness(disp_left_est, left_pyramid)
        disp_right_smoothness = self.disp_smoothness(disp_right_est, right_pyramid)

        # L1
        l1_left = [torch.mean(torch.abs(left_est[i] - left_pyramid[i])) for i in range(self.n)]
        l1_right = [torch.mean(torch.abs(right_est[i] - right_pyramid[i])) for i in range(self.n)]

        # SSIM
        ssim_left = [torch.mean(self.SSIM(left_est[i], left_pyramid[i])) for i in range(self.n)]
        ssim_right = [torch.mean(self.SSIM(right_est[i], right_pyramid[i])) for i in range(self.n)]

        image_loss_left = [self.SSIM_w * ssim_left[i] + (1 - self.SSIM_w) * l1_left[i] for i in range(self.n)]
        image_loss_right = [self.SSIM_w * ssim_right[i] + (1 - self.SSIM_w) * l1_right[i] for i in range(self.n)]
        image_loss = sum(image_loss_left + image_loss_right)

        # L-R Consistency
        lr_left_loss = [torch.mean(torch.abs(right_left_disp[i] - disp_left_est[i])) for i in range(self.n)]
        lr_right_loss = [torch.mean(torch.abs(left_right_disp[i] - disp_right_est[i])) for i in range(self.n)]
        lr_loss = sum(lr_left_loss + lr_right_loss)

        # Disparities smoothness
        disp_left_loss = [torch.mean(torch.abs(disp_left_smoothness[i])) / 2 ** i for i in range(self.n)]
        disp_right_loss = [torch.mean(torch.abs(disp_right_smoothness[i])) / 2 ** i for i in range(self.n)]
        disp_gradient_loss = sum(disp_left_loss + disp_right_loss)

        loss = image_loss + self.disp_gradient_w * disp_gradient_loss + self.lr_w * lr_loss
        self.image_loss = image_loss
        self.disp_gradient_loss = disp_gradient_loss
        self.lr_loss = lr_loss
        return loss


##**Utils**

In [0]:
def image_transforms(transformations=None,  size=(256, 512)):
        return transforms.Compose([ResizeImage(size=size), ToTensor()])


class ResizeImage(object):
    def __init__(self, size=(256, 512)):
        self.transform = transforms.Resize(size)

    def __call__(self, sample):
        left_image = sample['left_image']
        right_image = sample['right_image']
        new_right_image = self.transform(right_image)
        new_left_image = self.transform(left_image)
        sample = {'left_image': new_left_image, 'right_image': new_right_image}
        return sample


class ToTensor(object):
    def __init__(self):
        self.transform = transforms.ToTensor()

    def __call__(self, sample):
        left_image = sample['left_image']
        right_image = sample['right_image']
        new_right_image = self.transform(right_image)
        new_left_image = self.transform(left_image)
        sample = {'left_image': new_left_image, 'right_image': new_right_image}
        return sample


def to_device(input, device):
    if torch.is_tensor(input):
        return input.to(device=device)
    elif isinstance(input, str):
        return input
    elif isinstance(input, collections.Mapping):
        return {k: to_device(sample, device=device) for k, sample in input.items()}
    elif isinstance(input, collections.Sequence):
        return [to_device(sample, device=device) for sample in input]
    else:
        raise TypeError('Input must contain tensor, dict or list, found {type(input)}')

        
def prepare_dataloader(drive, shuffle, batch_size, size, num_workers):
    data_transform = image_transforms(size = size)
    
    frames_list = []
    for frame in os.listdir(drive + '/image_02/data/'): # add all frames in a sequence
        frames_list.append(drive + '/image_02/data/' + frame + ',' + drive + '/image_03/data/' + frame)
            
    dataset = KittiLoader(frames_list, transform = data_transform) # define dataset
    n_img = len(dataset) # get images number
    print('The drive contains', n_img, 'frames')
    loader = DataLoader(dataset, batch_size = batch_size, shuffle = shuffle, num_workers = num_workers, pin_memory = True) # create loader
    return n_img, loader


class KittiLoader(Dataset):
    def __init__(self, lines, transform=None):
        
        #with open(root_dir + '.txt') as file:
        #    lines = file.readlines()
        self.transform = transform
        self.lines = lines

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        left_image = Image.open(self.lines[idx].split(',')[0]) # get the left image [0]
        right_image = Image.open(self.lines[idx].split(',')[1]) # get the right image [1]
        sample = {'left_image': left_image, 'right_image': right_image}
        if self.transform: # apply a transformation if defined
            sample = self.transform(sample)
        return sample


def set_dataset_dict(test_dict):
    # read eigen_test and create a dictionary of drives and the list of frames to be evaluated for each one
    with open(test_split) as ts:
        for line in ts:
            frame = line.split()[0]
            if frame.split('/')[1] not in test_dict.keys():
                test_dict[frame.split('/')[1]] = []
            test_dict[frame.split('/')[1]].append(frame.split('/')[4])

##**Eval Utils**

In [0]:
def compute_errors(gt, pred):
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25   ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())

    d1 = np.mean(np.abs(gt - pred))

    rmse_log = (np.log(gt) - np.log(pred)) ** 2
    rmse_log = np.sqrt(rmse_log.mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)

    sq_rel = np.mean(((gt - pred)**2) / gt)

    return abs_rel, sq_rel, rmse, rmse_log, d1, a1, a2, a3


def read_calib_file(path):
    # taken from https://github.com/hunse/kitti
    float_chars = set('0123456789.e+- ')
    data = {}
    with open(path, 'r') as f:
        for line in f.readlines():
            key, value = line.split(':', 1)
            value = value.strip()
            data[key] = value
            if float_chars.issuperset(value):
                # try to cast to float array
                try:
                    data[key] = np.array(list(map(float, value.split(' '))))
                except ValueError:
                    # casting error: data[key] already eq. value, so pass
                    pass

    return data


def get_focal_length_baseline(calib_dir, cam):
    cam2cam = read_calib_file(calib_dir + 'calib_cam_to_cam.txt')
    P2_rect = cam2cam['P_rect_02'].reshape(3,4)
    P3_rect = cam2cam['P_rect_03'].reshape(3,4)

    # cam 2 is left of camera 0  -6cm
    # cam 3 is to the right  +54cm
    b2 = P2_rect[0,3] / -P2_rect[0,0]
    b3 = P3_rect[0,3] / -P3_rect[0,0]
    baseline = b3-b2

    if cam==2:
        focal_length = P2_rect[0,0]
    elif cam==3:
        focal_length = P3_rect[0,0]

    return focal_length, baseline
  
def eval_frame_disps(statistics, frame, kitti_path, drive, loss, disps, frame_idx):
    disp = disps[0][frame_idx, 0, :, :].unsqueeze(1) # (1,2,256,512) -> (1,256,512)
    disp = disp.squeeze().cpu().detach().numpy() # (1,256,512) -> (256,512)
    gt_depth = cv2.imread(kitti_path + drive.rsplit('_', 3)[0] + '/' + drive + '/' + gt_path + '/' + frame, cv2.IMREAD_UNCHANGED) # get groundtruth image (only left one)
    gt_depth = np.array(gt_depth, dtype=np.float32)
    gt_depth = gt_depth/256 # divide for 256 ?????????????????????????
    im_size = gt_depth.shape[:2] # (375, 1242)
    disp_pred = cv2.resize(disp, (im_size[1], im_size[0]), interpolation = cv2.INTER_LINEAR) # resize translated images to the groundtruth image size
    disp_pred = disp_pred * disp_pred.shape[1] # multiply for a scalar (1242) ????????????????????????????????
    camera_id = 2
    focal_length, baseline = get_focal_length_baseline(kitti_path + drive.rsplit('_', 3)[0] + '/', camera_id)
    depth_pred = (baseline * focal_length) / disp_pred # predicted depth
    depth_pred[np.isinf(depth_pred)] = 0 # set infinite values of prediction to 0
    min_depth = 1e-3
    max_depth = 50
    mask = np.logical_and(gt_depth > min_depth, gt_depth < max_depth) # create a mask to restrict the range of predicted depths to [0.001, 50]
    gt_height, gt_width = gt_depth.shape # eigen crop
    crop = np.array([0.3324324 * gt_height, 0.91351351 * gt_height, 0.0359477 * gt_width, 0.96405229 * gt_width]).astype(np.int32) # crop image with specific values ??????????
    crop_mask = np.zeros(mask.shape)
    crop_mask[crop[0]:crop[1],crop[2]:crop[3]] = 1
    mask = np.logical_and(mask, crop_mask) # build a matrix based on the mask
    abs_rel, sq_rel, rmse, log_rmse, d1, a1, a2, a3 = compute_errors(gt_depth[mask], depth_pred[mask]) # compute statistics with dept_pred and groundtruth using the mask
    # add statistics to respective vectors
    statistics[0]['loss'].append(loss.item()) # loss
    statistics[0]['abs_rel'].append(abs_rel) # relative absolute error
    statistics[0]['sq_rel'].append(sq_rel) # squared relative absolute error
    statistics[0]['rmse'].append(rmse) # root mean square error
    statistics[0]['log_rmse'].append(log_rmse) # log rmse
    statistics[0]['d1'].append(d1) # absolute error
    statistics[0]['a1'].append(a1) # diff<1.25
    statistics[0]['a2'].append(a2) # diff<1.25**2
    statistics[0]['a3'].append(a3) # diff<1.25**3
    statistics[1] += 1 # number of frames evaluated
      
    if show_logs:
        print('│    {:^4d}       {:^6.4f}     {:^7.4f}     {:^7.4f}     {:^7.4f}     {:^7.4f}     {:^7.4f}    {:^6.4f}    {:^6.4f}    {:^6.4f}  │'
               .format(int(frame.split('.')[0]), loss.item(), abs_rel, sq_rel, rmse, log_rmse, d1, a1, a2, a3))

##**Logs and Graphs**

In [0]:
def logs_and_graphs(statistics, drive):
    n = math.ceil(statistics[1]/5)
    if show_avg_logs:
        print('│  Averages     {:^6.4f}     {:^7.4f}     {:^7.4f}     {:^7.4f}     {:^7.4f}     {:^7.4f}    {:^6.4f}    {:^6.4f}    {:^6.4f}  │'
              .format(sum(statistics[0]['loss'])/statistics[1],
                      sum(statistics[0]['abs_rel'])/statistics[1],
                      sum(statistics[0]['sq_rel'])/statistics[1],
                      sum(statistics[0]['rmse'])/statistics[1],
                      sum(statistics[0]['log_rmse'])/statistics[1],
                      sum(statistics[0]['d1'])/statistics[1],
                      sum(statistics[0]['a1'])/statistics[1],
                      sum(statistics[0]['a2'])/statistics[1],
                      sum(statistics[0]['a3'])/statistics[1]))
        print('│  Last 20%     {:^6.4f}     {:^7.4f}     {:^7.4f}     {:^7.4f}     {:^7.4f}     {:^7.4f}    {:^6.4f}    {:^6.4f}    {:^6.4f}  │'
              .format(sum(statistics[0]['loss'][-n:])/n,
                      sum(statistics[0]['abs_rel'][-n:])/n,
                      sum(statistics[0]['sq_rel'][-n:])/n,
                      sum(statistics[0]['rmse'][-n:])/n,
                      sum(statistics[0]['log_rmse'][-n:])/n,
                      sum(statistics[0]['d1'][-n:])/n,
                      sum(statistics[0]['a1'][-n:])/n,
                      sum(statistics[0]['a2'][-n:])/n,
                      sum(statistics[0]['a3'][-n:])/n))
        print('└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘')
            
    if save_logs or save_graphs:
        new_folder = 'results/' + drive.rsplit('_', 3)[0] + '/' + drive + '/'
        if not os.path.exists(new_folder):
            os.makedirs(new_folder)

    if save_logs:
        # saves stats into log files
        with open(new_folder + 'log_loss.txt', 'w') as log:
            for stat in statistics[0]['loss']:
                log.write('%.4f\n' % stat)
        with open(new_folder + 'log_abs_rel.txt', 'w') as log:
            for stat in statistics[0]['abs_rel']:
                log.write('%.4f\n' % stat)
        with open(new_folder + 'log_sq_rel.txt', 'w') as log:
            for stat in statistics[0]['sq_rel']:
                log.write('%.4f\n' % stat)
        with open(new_folder + 'log_rmse.txt', 'w') as log:
            for stat in statistics[0]['rmse']:
                log.write('%.4f\n' % stat)
        with open(new_folder + 'log_log_rmse.txt', 'w') as log:
            for stat in statistics[0]['log_rmse']:
                log.write('%.4f\n' % stat)
        with open(new_folder + 'log_d1.txt', 'w') as log:
            for stat in statistics[0]['d1']:
                log.write('%.4f\n' % stat)
        with open(new_folder + 'log_a1.txt', 'w') as log:
            for stat in statistics[0]['a1']:
                log.write('%.4f\n' % stat)
        with open(new_folder + 'log_a2.txt', 'w') as log:
            for stat in statistics[0]['a2']:
                log.write('%.4f\n' % stat)
        with open(new_folder + 'log_a3.txt', 'w') as log:
            for stat in statistics[0]['a3']:
                log.write('%.4f\n' % stat)
    
    if show_graphs or save_graphs:
        graphs, subplt = plt.subplots(3, 3, figsize = (15, 15)) # 9 graphs, one for each statistic
        for (i, stat) in enumerate(statistics[0]):
            subplt[math.floor(i/3), i%3].plot(statistics[0][stat])
            subplt[math.floor(i/3), i%3].set_title(stat, color='white', fontsize=20)
            subplt[math.floor(i/3), i%3].grid(True)
            subplt[math.floor(i/3), i%3].tick_params(colors='white')
            subplt[math.floor(i/3), i%3].patch.set_facecolor('xkcd:very pale blue')
        if (save_graphs):
            plt.savefig(new_folder + 'graphs.png', facecolor = graphs.get_facecolor(), edgecolor='none')
        if (show_graphs):
            plt.show()
        plt.close()
    
    if show_each_sheet_string:
        print('=SPLIT("{:07.4f},{:07.4f},{:07.4f},{:07.4f},{:07.4f},{:07.4f},{:07.4f},{:07.4f}", ",")\n'
               .format(sum(statistics[0]['rmse'])/statistics[1],
                       sum(statistics[0]['abs_rel'])/statistics[1],
                       sum(statistics[0]['sq_rel'])/statistics[1],
                       sum(statistics[0]['log_rmse'])/statistics[1],
                       sum(statistics[0]['rmse'][-n:])/n,
                       sum(statistics[0]['abs_rel'][-n:])/n,
                       sum(statistics[0]['sq_rel'][-n:])/n,
                       sum(statistics[0]['log_rmse'][-n:])/n))
    
    return ('=SPLIT("{:07.4f},{:07.4f},{:07.4f},{:07.4f},{:07.4f},{:07.4f},{:07.4f},{:07.4f}", ",")\n'
            .format(sum(statistics[0]['rmse'])/statistics[1],
                    sum(statistics[0]['abs_rel'])/statistics[1],
                    sum(statistics[0]['sq_rel'])/statistics[1],
                    sum(statistics[0]['log_rmse'])/statistics[1],
                    sum(statistics[0]['rmse'][-n:])/n,
                    sum(statistics[0]['abs_rel'][-n:])/n,
                    sum(statistics[0]['sq_rel'][-n:])/n,
                    sum(statistics[0]['log_rmse'][-n:])/n))

##**Main**

In [0]:
# used to test a specific set of drives, remove the others form the dictionary
def edit_dataset_dict(dataset_dict):
    '''
    del dataset_dict['2011_09_26_drive_0002_sync'] # 77
    del dataset_dict['2011_09_26_drive_0009_sync'] # 447
    del dataset_dict['2011_09_26_drive_0013_sync'] # 144
    del dataset_dict['2011_09_26_drive_0020_sync'] # 86
    del dataset_dict['2011_09_26_drive_0023_sync'] # 474
    del dataset_dict['2011_09_26_drive_0027_sync'] # 188
    del dataset_dict['2011_09_26_drive_0029_sync'] # 430
    del dataset_dict['2011_09_26_drive_0036_sync'] # 803
    del dataset_dict['2011_09_26_drive_0046_sync'] # 125
    del dataset_dict['2011_09_26_drive_0048_sync'] # 22
    del dataset_dict['2011_09_26_drive_0052_sync'] # 78
    del dataset_dict['2011_09_26_drive_0056_sync'] # 294
    del dataset_dict['2011_09_26_drive_0059_sync'] # 373
    del dataset_dict['2011_09_26_drive_0064_sync'] # 570
    del dataset_dict['2011_09_26_drive_0084_sync'] # 383
    del dataset_dict['2011_09_26_drive_0086_sync'] # 706
    del dataset_dict['2011_09_26_drive_0093_sync'] # 433
    del dataset_dict['2011_09_26_drive_0096_sync'] # 475
    del dataset_dict['2011_09_26_drive_0101_sync'] # 936
    del dataset_dict['2011_09_26_drive_0106_sync'] # 227
    del dataset_dict['2011_09_26_drive_0117_sync'] # 660
    del dataset_dict['2011_09_28_drive_0002_sync'] # 376
    del dataset_dict['2011_09_29_drive_0071_sync'] # 1059
    del dataset_dict['2011_09_30_drive_0016_sync'] # 279
    del dataset_dict['2011_09_30_drive_0018_sync'] # 2762
    del dataset_dict['2011_09_30_drive_0027_sync'] # 1106
    del dataset_dict['2011_10_03_drive_0027_sync'] # 4544
    del dataset_dict['2011_10_03_drive_0047_sync'] # 837
    '''

In [0]:
# parameters definition
#pre_trained_model = 'drive/My Drive/pretrained_model_197_mine.pth' # path of the pretrained model
#pre_trained_model = 'drive/My Drive/pretrained_model_119_mine.pth' # path of the pretrained model
pre_trained_model = 'drive/My Drive/pretrained_model_zhenyu.pth' # path of the pretrained model
#pre_trained_model = 'drive/My Drive/pretrained_model_switchnorm_124.pth' # path of the pretrained model
kitti_path = 'drive/My Drive/Kitti/' # path of the dataset
test_split = kitti_path + 'eigen_test.txt' # list of fraes to evaluate
gt_path = 'proj_depth/groundtruth/image_02' # path of the groundtruth images, used during evaluation
input_height = 256
input_width = 512
input_channels = 3*2 # rbg * two images
model = 'resnet50_md'
#model = 'resnet50_md_switchorm'
batch_size = 1
shuffle = False
learning_rate = 1e-4
bn_momentum = 0.03 # momentum for the batch normalization layers
device = 'cuda:0'
num_workers = 0

# 0 - NAIVE
# 1 - Meta-Learning
# 2 - OFDA
# 3 - OMLA
used_method = 2

show_logs = True # shows logs of each frame
show_avg_logs = True # shows logs of average score of each drive
show_graphs = True # shows 9 graphs of the statistics of the drive
show_each_sheet_string = True # show a string to paste on google sheets to copy results for an individual drive
show_sheet_string = True # show a string to paste on google sheets to copy results for all drives
save_logs = True # save logs on results/<drive_day>/<drive>/log_<stat_name>.txt
save_graphs = True # save the graphs in results/<drive_day>/<drive>/graphs.png


class Model:

    def __init__(self):
        # Set up model
        self.device = device
        self.model = Resnet50_md(input_channels)
        self.model = self.model.to(self.device)

        self.loss_function = MonodepthLoss(n = 4, SSIM_w = 0.85, disp_gradient_w = 0.1, lr_w = 1).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr = learning_rate)
        
        self.input_height = input_height
        self.input_width = input_width
        self.dataset_dict = {}
        self.transform = transforms.Compose([transforms.Resize((input_height, input_width)), transforms.ToTensor()])
                
        self.statistics = [{'loss': [], 'rmse':[], 'log_rmse':[], 'abs_rel':[], 'sq_rel':[], 'd1':[], 'a1':[], 'a2':[], 'a3':[]}, 0]
        
        if 'cuda' in self.device:
            torch.cuda.synchronize()
    
    def perform_domain_adaptation(self):
        if used_method != 0 and used_method != 1 and used_method != 2 and used_method != 3:
            print('Invalid method selected')
        else:
            set_dataset_dict(self.dataset_dict)
            edit_dataset_dict(self.dataset_dict)
            sheet_string = ''
            
            if used_method == 0:
                print('Currently using Onine Naive method')
            elif used_method == 1:
                print('Currently using Meta Learning method')
            elif used_method == 2:
                print('Currently using OFDA method')
            elif used_method == 3:
                print('Currently using OMLA method')

            print('Starting with learning rate =', learning_rate)
            
            for module in self.model.modules():
                if isinstance(module, nn.BatchNorm2d):
                    module.momentum = bn_momentum # set the momentum of a batch normalization layer to the desired value

            print('Momentum of BN Layers =', bn_momentum)
            
            for drive in self.dataset_dict.keys():
                print('═══════════════════════════════════════ drive : ' + drive + ' ════════════════════════════════════════')
                print()

                drive_frames = os.listdir(kitti_path + drive.rsplit('_', 3)[0] + '/' + drive + '/image_02/data/')
                
                if shuffle:
                    random.shuffle(drive_frames)
                    for kf in self.dataset_dict[drive]:
                        drive_frames[drive_frames.index(kf)], drive_frames[int(kf.split('.')[0])] = drive_frames[int(kf.split('.')[0])], drive_frames[drive_frames.index(kf)]
                else:
                    drive_frames.sort()
                
                self.statistics = [{'loss': [], 'rmse':[], 'log_rmse':[], 'abs_rel':[], 'sq_rel':[], 'd1':[], 'a1':[], 'a2':[], 'a3':[]}, 0]
                print('Loading trained network ' + pre_trained_model)
                print('This drive contains ' + str(len(drive_frames)) + ' frames')
                
                # OLDER VERSION OF TORCH/TORCHVISION
                #model_dict = self.model.state_dict()
                #pretrained_dict = {k: v for k, v in torch.load(pre_trained_model).items() if k in model_dict}
                # 2. overwrite entries in the existing state dict
                #model_dict.update(pretrained_dict) 
                # 3. load the new state dict
                #self.model.load_state_dict(pretrained_dict)
                self.model.load_state_dict(torch.load(pre_trained_model))
      
                if show_logs or show_avg_logs:
                    print()
                    print('┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐')
                    print('│    Frame       Loss      Abs_rel      Sq_rel      Rmse      Rmse_Log       d1         a1        a2        a3    │')
                    print('├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤')
                    
                for i in range(0, len(drive_frames), batch_size):
                    # prepare batch
                    left_imgs = []
                    right_imgs = []
                    for frame in drive_frames[i:i+batch_size]:
                        left_imgs.append(self.transform(Image.open(kitti_path + drive.rsplit('_', 3)[0] + '/' + drive + '/image_02/data/' + frame)))
                        right_imgs.append(self.transform(Image.open(kitti_path + drive.rsplit('_', 3)[0] + '/' + drive + '/image_03/data/' + frame)))
                    left = to_device(torch.stack(left_imgs, dim = 0), device)
                    right = to_device(torch.stack(right_imgs, dim = 0), device)
                    img_pair = torch.cat((left, right), 1)
                    self.optimizer.zero_grad()
                    
                    # 0 : Naive - No BN statistics update and fixed learning rate (no global statistics)
                    # 1 : Meta Learning - No BN statistics update and meta learning rate
                    # 2 : OFDA - BN statistics update and fixed learning rate
                    # 3 : OMLA - BN statistics update and meta learning rate

                    # BN statistics update (only OFDA and OMLA)
                    if used_method == 2 or used_method == 3:
                        self.model.train()
                        with torch.no_grad(): # disable gradient computation for the forward pass
                            dispsf = self.model(img_pair, True) # execute forward pass to update running mean and variance of BatchNorm layer
                    
                    # online naive use train mode to avoid the usage of collected BN statistic
                    if used_method == 0:
                        self.model.train()
                    else:
                        self.model.eval()

                    # get model output and self supervised loss
                    disps = self.model(img_pair, False) # use model to obtain disparities
                    loss = self.loss_function(disps, [left, right]) # calculate self-supervised loss from the images and the disparities
                    
                    # meta learning rate update (only Meta Learning and OMLA)
                    #if used_method == 1 or used_method == 3:
                        # update learning rate for each parameter with a gradient descent step

                    # network weights update and evaluation
                    loss.backward() # calculate gradients updates with the self-supervised loss
                    self.optimizer.step() # update network parameters
                    
                    # evaluation on groundtruth depth only for frames in eigen_test split
                    for i,frame in enumerate (drive_frames[i:i+batch_size]):
                        if frame in self.dataset_dict[drive]:
                            if os.path.isfile(kitti_path + drive.rsplit('_', 3)[0] + '/' + drive + '/' + gt_path + '/' + frame):
                                eval_frame_disps(self.statistics, frame, kitti_path, drive, loss, disps, i)
                
                if show_logs:
                    print('├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤')
                sheet_string += logs_and_graphs(self.statistics, drive)
            
            if show_sheet_string:
                print()
                print('Copy and paste on Google Sheets (Ctrl+Shift+V to avoid style copy)')
                sheet_string = sheet_string.rsplit('\n', 1)[0] # remove last crlf
                print(sheet_string)


def main():
    model = Model()
    model.perform_domain_adaptation()

    
if __name__ == '__main__':
    main()



In [0]:
!zip -r results.zip results

##**Extra**

In [0]:
# compute averages of performances for first 20 frames adabn on-off test
avg_statistics = [{'loss':     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                   'rmse':     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                   'log_rmse': [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                   'abs_rel':  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                   'sq_rel':   [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                   'd1':       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                   'a1':       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                   'a2':       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                   'a3':       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}, 0]

for day in os.listdir('results'):
    for drive in os.listdir('results/' + day):
        avg_statistics[1] += 1
        for file in os.listdir('results/' + day + '/' + drive):
            if file != 'graphs.png':
                with open('results/' + day + '/' + drive + '/' + file) as f:
                    for i,line in enumerate(f):
                        avg_statistics[0][file.split('_',1)[1].split('.')[0]][i] += float(line)

avg_statistics[1] = avg_statistics[1] * 15
for i in range(15):
    for stat in avg_statistics[0]:
        print(i+5, stat, avg_statistics[0][stat][i]/avg_statistics[1])
    print()

In [0]:
#cholensky decomposition test

mean = [2.5,1,0]
var = [0.5,0.2,1]
#emp_cov = np.cov()
emp_cov = [[4,12,-16],
           [12,37,-43],
           [-16,-43,98]]
print(emp_cov)
wb = np.linalg.cholesky(emp_cov)
print(wb)